In [1]:
import requests
import pandas as pd
import json
from path import Path
from dotenv import load_dotenv
import os
import requests
import datanews
from datetime import datetime
import asyncio
import time
import elsapy
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json

In [4]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

In [6]:
## Initialize client
client = ElsClient(config['apikey'])

In [7]:
## Author example
# Initialize author with uri
my_auth = ElsAuthor(
        uri = 'https://api.elsevier.com/content/author/author_id/7004367821')
# Read author data, then write to disk
if my_auth.read(client):
    print ("my_auth.full_name: ", my_auth.full_name)
    my_auth.write()
else:
    print ("Read author failed.")

my_auth.full_name:  Paul John McGuiness


In [8]:
## Affiliation example
# Initialize affiliation with ID as string
my_aff = ElsAffil(affil_id = '60101411')
if my_aff.read(client):
    print ("my_aff.name: ", my_aff.name)
    my_aff.write()
else:
    print ("Read affiliation failed.")

my_aff.name:  American Institute of Steel Construction


In [9]:
## Scopus (Abtract) document example
# Initialize document with ID as integer
scp_doc = AbsDoc(scp_id = 84872135457)
if scp_doc.read(client):
    print ("scp_doc.title: ", scp_doc.title)
    scp_doc.write()   
else:
    print ("Read document failed.")

scp_doc.title:  Control of somatic tissue differentiation by the long non-coding RNA TINCR


In [10]:
## ScienceDirect (full-text) document example using PII
pii_doc = FullDoc(sd_pii = 'S1674927814000082')
if pii_doc.read(client):
    print ("pii_doc.title: ", pii_doc.title)
    pii_doc.write()   
else:
    print ("Read document failed.")

pii_doc.title:  Establishing the fair allocation of international aviation carbon emission rights


In [11]:
## ScienceDirect (full-text) document example using DOI
doi_doc = FullDoc(doi = '10.1016/S1525-1578(10)60571-5')
if doi_doc.read(client):
    print ("doi_doc.title: ", doi_doc.title)
    doi_doc.write()   
else:
    print ("Read document failed.")

doi_doc.title:  Sensitive Sequencing Method for KRAS Mutation Detection by Pyrosequencing


In [12]:
## Load list of documents from the API into affilation and author objects.
# Since a document list is retrieved for 25 entries at a time, this is
#  a potentially lenghty operation - hence the prompt.
print ("Load documents (Y/N)?")
s = input('--> ')

if (s == "y" or s == "Y"):

    ## Read all documents for example author, then write to disk
    if my_auth.read_docs(client):
        print ("my_auth.doc_list has " + str(len(my_auth.doc_list)) + " items.")
        my_auth.write_docs()
    else:
        print ("Read docs for author failed.")

    ## Read all documents for example affiliation, then write to disk
    if my_aff.read_docs(client):
        print ("my_aff.doc_list has " + str(len(my_aff.doc_list)) + " items.")
        my_aff.write_docs()
    else:
        print ("Read docs for affiliation failed.")

Load documents (Y/N)?


-->  n


In [34]:
## Initialize doc search object using ScienceDirect and execute search, 
#   retrieving all results
doc_srch = ElsSearch("Nanosafety",'sciencedirect')
doc_srch.execute(client)
print ("doc_srch has", len(doc_srch.results), "results.")

doc_srch has 25 results.


In [35]:
# Set file path
with open('dump.json', 'r') as json_file:
    dump = json.load(json_file)

dump[0]

{'@_fa': 'true',
 'load-date': '2020-11-01T00:00:00.000Z',
 'link': [{'@_fa': 'true',
   '@ref': 'self',
   '@href': 'https://api.elsevier.com/content/article/pii/S1748013220301791'},
  {'@_fa': 'true',
   '@ref': 'scidir',
   '@href': 'https://www.sciencedirect.com/science/article/pii/S1748013220301791?dgcid=api_sd_search-api-endpoint'}],
 'dc:identifier': 'DOI:10.1016/j.nantod.2020.101010',
 'prism:url': 'https://api.elsevier.com/content/article/pii/S1748013220301791',
 'dc:title': 'Nanosafety evaluation through feces: A comparison between selenium nanoparticles and selenite in rats',
 'dc:creator': 'Xiaoying Lin',
 'prism:publicationName': 'Nano Today',
 'prism:volume': '36',
 'prism:coverDate': '2021-02-28',
 'prism:startingPage': '101010',
 'prism:doi': '10.1016/j.nantod.2020.101010',
 'openaccess': False,
 'pii': 'S1748013220301791',
 'authors': {'author': [{'$': 'Xiaoying Lin'},
   {'$': 'Liming Wang'},
   {'$': 'Yu-Feng Li'}]}}

In [36]:
df = pd.DataFrame.from_dict(dump)
df

,@_fa,authors,dc:creator,dc:identifier,dc:title,link,load-date,openaccess,pii,prism:coverDate,prism:doi,prism:endingPage,prism:publicationName,prism:startingPage,prism:url,prism:volume
0,true,"{'author': [{'$': 'Xiaoying Lin'}, {'$': 'Limi...",Xiaoying Lin,DOI:10.1016/j.nantod.2020.101010,Nanosafety evaluation through feces: A compari...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2020-11-01T00:00:00.000Z,False,S1748013220301791,2021-02-28,10.1016/j.nantod.2020.101010,NaN,Nano Today,101010,https://api.elsevier.com/content/article/pii/S...,36
1,true,"{'author': [{'$': 'Vicenç Pomar-Portillo'}, {'...",Vicenç Pomar-Portillo,DOI:10.1016/j.impact.2021.100323,Nanosafety research in Europe – Towards a focu...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2021-05-15T00:00:00.000Z,False,S245207482100032X,2021-04-30,10.1016/j.impact.2021.100323,NaN,NanoImpact,100323,https://api.elsevier.com/content/article/pii/S...,22
2,true,"{'author': [{'$': 'Ali Ayadi'}, {'$': 'Jérôme ...",Ali Ayadi,DOI:10.1016/j.impact.2020.100288,MESOCOSM: A mesocosm database management syste...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2020-12-22T00:00:00.000Z,False,S2452074820300823,2021-01-31,10.1016/j.impact.2020.100288,NaN,NanoImpact,100288,https://api.elsevier.com/content/article/pii/S...,21
3,true,"{'author': [{'$': 'Dorsa Parviz'}, {'$': 'Dimi...",Dorsa Parviz,DOI:10.1016/j.impact.2020.100226,Engineering two-dimensional nanomaterials to e...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2020-04-18T00:00:00.000Z,False,S2452074820300203,2020-04-30,10.1016/j.impact.2020.100226,NaN,NanoImpact,100226,https://api.elsevier.com/content/article/pii/S...,18
4,true,"{'author': [{'$': 'Aline M. Z. de Medeiros'}, ...",Aline M. Z. de Medeiros,DOI:10.1016/j.ecoenv.2020.111776,Graphene oxide-silver nanoparticle hybrid mate...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2020-12-17T00:00:00.000Z,True,S0147651320316122,2021-02-28,10.1016/j.ecoenv.2020.111776,NaN,Ecotoxicology and Environmental Safety,111776,https://api.elsevier.com/content/article/pii/S...,209
5,true,"{'author': [{'$': 'Chengyong He'}, {'$': 'Shen...",Chengyong He,DOI:10.1016/j.nano.2018.09.003,High-content analysis for mitophagy response t...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2018-09-20T00:00:00.000Z,False,S1549963418305215,2019-01-31,10.1016/j.nano.2018.09.003,69,"Nanomedicine: Nanotechnology, Biology and Medi...",59,https://api.elsevier.com/content/article/pii/S...,15
6,true,"{'author': [{'$': 'Natalie Burden'}, {'$': 'Ka...",Natalie Burden,DOI:10.1016/j.nantod.2016.06.007,The 3Rs as a framework to support a 21st centu...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2016-07-25T00:00:00.000Z,True,S1748013215301079,2017-02-28,10.1016/j.nantod.2016.06.007,13,Nano Today,10,https://api.elsevier.com/content/article/pii/S...,12
7,true,"{'author': [{'$': 'Antreas Afantitis'}, {'$': ...",Antreas Afantitis,DOI:10.1016/j.csbj.2020.02.023,NanoSolveIT Project: Driving nanoinformatics r...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2020-03-07T00:00:00.000Z,True,S2001037019305112,2020-12-31,10.1016/j.csbj.2020.02.023,602,Computational and Structural Biotechnology Jou...,583,https://api.elsevier.com/content/article/pii/S...,18
8,true,"{'author': [{'$': 'Andrea Haase'}, {'$': 'Iseu...",Andrea Haase,DOI:10.1016/j.impact.2018.02.005,Quality in nanosafety — Towards reliable nanom...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2018-02-15T00:00:00.000Z,False,S2452074818300181,2018-07-31,10.1016/j.impact.2018.02.005,68,NanoImpact,67,https://api.elsevier.com/content/article/pii/S...,11
9,true,"{'author': [{'$': 'Md. Ershadul Karim'}, {'$':...",Md. Ershadul Karim,DOI:10.1016/j.techfore.2014.07.003,Too enthusiastic to care for safety: Present s...,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",2014-08-12T00:00:00.000Z,False,S0040162514002285,2015-03-31,10.1016/j.techfore.2014.07.003,181,Technological Forecasting and Social Change,168,https://api.el